In [ ]:
!gcloud components update

In [ ]:
import torch
from torch.autograd import Variable

print 'PyTorch Version: {}'.format(torch.__version__)

In [ ]:
%%bash

mkdir data
mkdir saved_models

In [ ]:
DATA_FILE = "data/iris.csv"
MODEL_DIR = "saved_models/model.joblib"

## Download Iris Data

In [ ]:
import urllib

url_opener = urllib.URLopener()
url_opener.retrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", DATA_FILE)

## Load Data to Pandas Dataframes

In [ ]:
import pandas as pd

datatrain = pd.read_csv(DATA_FILE, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

#change string value to numeric
datatrain.loc[datatrain['species']=='Iris-setosa', 'species']=0
datatrain.loc[datatrain['species']=='Iris-versicolor', 'species']=1
datatrain.loc[datatrain['species']=='Iris-virginica', 'species']=2
datatrain = datatrain.apply(pd.to_numeric)

#change dataframe to array
datatrain_array = datatrain.as_matrix()

#split x and y (feature and target)
xtrain = datatrain_array[:,:4]
ytrain = datatrain_array[:,4]

print len(xtrain)

## Model Parameters

In [ ]:
input_features = 4
hidden_units = 10
num_classes = 3
learning_rate = 0.1
momentum = 0.9
num_epoch = 10000

## Model Definition

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(input_features, hidden_units),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_units, num_classes),
    torch.nn.Softmax()
)

loss_metric = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=momentum)

## Model Training

In [ ]:
for epoch in range(num_epoch):
    
    x = Variable(torch.Tensor(xtrain).float())
    y = Variable(torch.Tensor(ytrain).long())

    optimizer.zero_grad()
    
    y_pred = model(x)
    loss = loss_metric(y_pred, y)

    loss.backward()
    optimizer.step()

    if (epoch) % 1000 == 0:
        print 'Epoch [{}/{}] Loss: {}'.format(epoch+1, num_epoch, round(loss.item(),3))
        
print 'Epoch [{}/{}] Loss: {}'.format(epoch+1, num_epoch, round(loss.item(),3))

## Save and Load the Model

In [ ]:
torch.save(model, MODEL_DIR)

In [ ]:
iris_classifier = torch.load(MODEL_DIR)

## Test Model for Predictions

In [ ]:
def predict_class(instances, vocab):
    instances = torch.Tensor(instances)
    output = iris_classifier(instances)
    _ , predicted = torch.max(output, 1)
    return [vocab[class_index] for class_index in predicted]

In [ ]:
print predict_class(xtrain[0:10], ['setosa', 'versicolor', 'virginica'])

## Upload Model to Cloud Storage

In [1]:
import os

PROJECT = 'ksalama-gcp-playground'
BUCKET = 'ksalama-gcs-cloudml'
REGION = 'europe-west1'
MODEL_NAME = 'torch_iris_classifier'
VERSION_NAME = 'v1'

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['VERSION_NAME'] = VERSION_NAME

In [ ]:
!gcloud config set project {PROJECT}

In [ ]:
!gsutil -m rm -r gs://${BUCKET}/models/pytorch

In [ ]:
%%bash

saved_model_dir="saved_models/model.joblib"
echo ${saved_model_dir}

gsutil -m cp -r ${saved_model_dir} gs://${BUCKET}/models/pytorch/iris_classifier/
gsutil ls gs://${BUCKET}/models/pytorch/iris_classifier

## Define the Custom Model Class

In [2]:
%%writefile model.py

import os
import pandas as pd
from google.cloud import storage
import torch


class PyTorchIrisClassifier(object):
    
    def __init__(self, model):
        self._model = model
        self.class_vocab = ['setosa', 'versicolor', 'virginica']
        
    @classmethod
    def from_path(cls, model_dir):
        model_file = os.path.join(model_dir,'model.joblib')
        model = torch.load(model_file)    
        return cls(model)

    def predict(self, instances, **kwargs):
        data = pd.DataFrame(instances).as_matrix()
        inputs = torch.Tensor(data)
        outputs = self._model(inputs)
        _ , predicted = torch.max(outputs, 1)
        return [self.class_vocab[class_index] for class_index in predicted]

Overwriting model.py


In [3]:
import model

model = model.PyTorchIrisClassifier.from_path('.')
model.predict([[1,2,3,4],[4,3,2,1]])

/Users/khalidsalama/Technology/python-venvs/py27-venv/lib/python2.7/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


['virginica', 'setosa']

In [4]:
%%writefile setup.py

from setuptools import setup

REQUIRED_PACKAGES = [
    'torch',
    #'google-cloud-storage',
    #'pandas'
]

setup(
    name="pytorch_iris",
    author="Khalid Salama",
    author_email="khalidsalama@google.com",
    version="0.1",
    scripts=["model.py"],
    install_requires=REQUIRED_PACKAGES
)

Overwriting setup.py


## Create a package 

In [5]:
!python setup.py sdist

running sdist
running egg_info
writing requirements to pytorch_iris.egg-info/requires.txt
writing pytorch_iris.egg-info/PKG-INFO
writing top-level names to pytorch_iris.egg-info/top_level.txt
writing dependency_links to pytorch_iris.egg-info/dependency_links.txt
reading manifest file 'pytorch_iris.egg-info/SOURCES.txt'
writing manifest file 'pytorch_iris.egg-info/SOURCES.txt'

running check

creating pytorch_iris-0.1
creating pytorch_iris-0.1/pytorch_iris.egg-info
copying files to pytorch_iris-0.1...
copying model.py -> pytorch_iris-0.1
copying setup.py -> pytorch_iris-0.1
copying pytorch_iris.egg-info/PKG-INFO -> pytorch_iris-0.1/pytorch_iris.egg-info
copying pytorch_iris.egg-info/SOURCES.txt -> pytorch_iris-0.1/pytorch_iris.egg-info
copying pytorch_iris.egg-info/dependency_links.txt -> pytorch_iris-0.1/pytorch_iris.egg-info
copying pytorch_iris.egg-info/requires.txt -> pytorch_iris-0.1/pytorch_iris.egg-info
copying pytorch_iris.egg-info/top_level.txt -> pytorch_iris-0.1/pytorch_iris.

In [6]:
!gsutil cp ./dist/model_pkg-0.1.tar.gz gs://{BUCKET}/models/pytorch/packages/iris-custom-model-0.1.tar.gz

Copying file://./dist/model_pkg-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      


## Create Model in Cloud MLE

In [7]:
!gcloud ml-engine models create {MODEL_NAME} --regions {REGION}
!echo ''
!gcloud ml-engine models list

ERROR: (gcloud.ml-engine.models.create) Resource in project [ksalama-gcp-playground] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name

NAME                    DEFAULT_VERSION_NAME
babyweight_estimator    v1
car_damages             car_damages_201805281633_base
census_classifier       v2
census_estimator        v1
housing_estimator       v1
iris_estimator          v1
mnist_classifier        v_org
synth_classifier        v1
synth_custom_regressor  v1
taxifare_estimator      v1
torch_iris_classifier   v1


In [8]:
!gcloud ml-engine versions delete {VERSION_NAME} --model={MODEL_NAME} --quiet
!echo ''
!gcloud ml-engine versions list --model {MODEL_NAME}

Deleting version [v1]......done.                                               

Listed 0 items.


In [ ]:
%%bash

MODEL_NAME='torch_iris_classifier'
VERSION_NAME='v1'
BUCKET='ksalama-gcs-cloudml'
MODEL_DIR='models/pytorch/iris_classifier'
PACKAGES_DIR='models/pytorch/packages'
RUNTIME_VERSION='1.10'
MODEL_CLASS='model.PyTorchIrisClassifier'

gcloud alpha ml-engine versions create ${VERSION_NAME} --model=${MODEL_NAME} \
            --origin=gs://${BUCKET}/${MODEL_DIR} \
            --runtime-version=${RUNTIME_VERSION} \
            --python-version=2.7 \
            --package-uris=gs://${BUCKET}/${PACKAGES_DIR}/iris-custom-model-0.1.tar.gz \
            --model-class=${MODEL_CLASS}

## ML Engine Online Prediction

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
                      discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')


def estimate(project, model_name, version, instances):
    request_data = {'instances': instances}

    model_url = 'projects/{}/models/{}/versions/{}'.format(project, model_name, version)
    response = api.projects().predict(body=request_data, name=model_url).execute()

    print response
    predictions = response["predictions"]

    return predictions

In [ ]:
PROJECT='ksalama-gcp-playground'
MODEL_NAME='torch_iris_classifier'
VERSION='v1'

instances = [
    [6.8, 2.8, 4.8, 1.4],
    [6. , 3.4, 4.5, 1.6]
]

predictions = estimate(instances=instances
                     ,project=PROJECT
                     ,model_name=MODEL_NAME
                     ,version=VERSION)

print(predictions)